In [1]:
import os

import numpy as np
import torchvision.transforms as T

import model.transforms as transforms

import utils.utils_mediapipe as utils_mediapipe
import utils.utils_plotly as utils_plotly
from config import DATA_CONFIG, TRAIN_CONFIG, VISUALIZER_CONFIG

In [2]:
def get_mp_graph(points: np.ndarray) -> np.ndarray:
    graph = np.array([
        8, 6, 5, 4, 0, 1, 2, 3, 7, None,
        10, 9, None,
        12, 11, 23, 24, 12, None,
        12, 14, 16, None,
        11, 13, 15, None,
        22, 16, 18, 20, 16, None,
        21, 15, 17, 19, 15, None,
        24, 26, 28, 30, 32, 28, None,
        23, 25, 27, 29, 31, 27, None,
    ])

    graphed_points = np.zeros((len(graph), 3))
    for i, node in enumerate(graph):
        if node is None:
            graphed_points[i] = np.nan
        else:
            graphed_points[i] = points[node]
    return graphed_points


def get_mp_graph_reduced(points: np.ndarray) -> np.ndarray:
    graph = np.array([
        12-11, 11-11, 23-11, 24-11, 12-11, None,
        12-11, 14-11, 16-11, None,
        11-11, 13-11, 15-11, None,
        22-11, 16-11, 18-11, 20-11, 16-11, None,
        21-11, 15-11, 17-11, 19-11, 15-11, None,
    ])

    graphed_points = np.zeros((len(graph), 3))
    for i, node in enumerate(graph):
        if node is None:
            graphed_points[i] = np.nan
        else:
            graphed_points[i] = points[node]
    return graphed_points

In [11]:
def get_skeleton_data(points: np.ndarray):
    mp_scatter = utils_plotly.get_scatter_3d(
        points,
        # points_colors,
        size=3,
    )

    mp_graph = utils_plotly.get_scatter_3d(
        get_mp_graph(points) if points.shape[0] == 33 else get_mp_graph_reduced(points),
        # points_colors,
        mode='lines',
        line=dict(color='darkblue', width=2),
    )

    return [mp_scatter, mp_graph]


def vis_data(data, file_path = ''):
    min_x, min_y, min_z = 0, 0, 0
    max_x, max_y, max_z = 0, 0, 0
    for item in data:
        min_x = min(min_x, item.x.min())
        min_y = min(min_y, item.y.min())
        min_z = min(min_z, item.z.min())
        max_x = max(max_x, item.x.max())
        max_y = max(max_y, item.y.max())
        max_z = max(max_z, item.z.max())
    mean_x = (min_x + max_x) / 2
    mean_y = (min_y + max_y) / 2
    mean_z = (min_z + max_z) / 2
    max_range = max(max_x - min_x, max_y - min_y, max_z - min_z)
    max_range /= 2

    fig = utils_plotly.create_figure_3d(len(data), **dict(data=data))
    fig.update_layout(
        title=file_path,
        scene_camera=VISUALIZER_CONFIG.scene_camera,
        scene=dict(
            xaxis=dict(range=[mean_x - max_range, mean_x + max_range]),
            yaxis=dict(range=[mean_y - max_range, mean_y + max_range]),
            zaxis=dict(range=[mean_z - max_range, mean_z + max_range]),
            aspectratio=dict(x=1, y=1, z=1),
        ),
    )
    fig.show()

In [12]:
device = 'cpu'

samples_folder = DATA_CONFIG.mediapipe.points_pose_world_windowed_filtered_labeled

label_map = TRAIN_CONFIG.gesture_set.label_map
inv_label_map = TRAIN_CONFIG.gesture_set.inv_label_map

In [13]:
to_keep = TRAIN_CONFIG.transforms_params.to_keep
shape_limit = TRAIN_CONFIG.transforms_params.shape_limit

train_transforms = transforms.TrainTransforms(
    to_keep=to_keep,
    shape_limit=shape_limit,
    device=device,
)
test_transforms = transforms.TestTransforms(
    to_keep=to_keep,
    shape_limit=shape_limit,
    device=device,
)

In [14]:
subject = 101
gesture = 'select'
hand = 'left'
trial = 1
frame = 0

file_path = os.path.join(
    samples_folder,
    f'G{subject}_{gesture}_{hand}_trial{trial}.npy'
)

data = utils_mediapipe.get_mediapipe_points(file_path)

Init data

In [15]:
transform = T.Compose([
    
])

points = np.copy(data[frame, :-1])
points = transform(points[None, ...]).reshape(-1, 3)

scatters = get_skeleton_data(points)
vis_data(scatters, file_path)

Transformed

In [16]:
transform = T.Compose([
    transforms.FilterIndex(to_keep),
    transforms.NumpyToTensor(device=device),
    transforms.NormalizePoints(dim=1),
    transforms.NormalRandom(std=0.1)
])

points = np.copy(data[frame, :-1])
points = transform(points[None, ...]).reshape(-1, 3)

scatters = get_skeleton_data(points)
vis_data(scatters, file_path)

Animation

In [17]:
transform = T.Compose([
    transforms.NumpyToTensor(device=device),
    transforms.NormalizePoints(dim=1),
    # transforms.FilterIndex(to_keep),
])

go_frames = []
for frame in range(len(data)):
    points = np.copy(data[frame, :-1])
    points = transform(points[None, ...]).reshape(-1, 3)

    scatters = get_skeleton_data(points)
    go_frame = utils_plotly.get_frame(data=scatters, frame_num=frame)
    go_frames.append(go_frame)

fig = utils_plotly.create_figure_3d(2)
fig.update(frames=go_frames)
fig.update_layout(
    title=file_path,
    scene=dict(
        xaxis=dict(range=[-3, 3]),
        yaxis=dict(range=[-3, 3]),
        zaxis=dict(range=[-3, 3]),
        aspectratio=dict(x=1, y=1, z=1),
    ),
    scene_camera=VISUALIZER_CONFIG.scene_camera,
    updatemenus=[VISUALIZER_CONFIG.update_buttons],
    uirevision=True,
)
fig.show(renderer='browser')